In [ ]:

import os
import numpy as np
import torch
import h5py
from model import LSTM_ManyToOne
from utils import calc_accuracy,eval_help,eval_variable_help
from dataloader import Data_Query
import parameters as par
import faiss
import tqdm
torch.cuda.set_device(1)


In [ ]:
with open(os.path.join(par.ROOT_DIR, "splits/Shopping100k/imgs_test.txt")) as f:
            img_data = f.read().splitlines()

In [ ]:

gallery_feat=np.load(par.FEAT_TEST_SENZA_N)
test_labels = np.loadtxt(os.path.join(par.ROOT_DIR,par.LABEL_TEST), dtype=int)
path=par.DATA_TEST
Data_test = h5py.File(path)
t_id=Data_test['t']#id del target 
query_labels=test_labels[t_id]
test_data=Data_Query(Data_test=Data_test,gallery_feat=gallery_feat,label_data=test_labels,N=par.N)
gallery_loader=torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False,
                                        sampler=torch.utils.data.SequentialSampler(test_data),
                                            drop_last=False)

model=LSTM_ManyToOne(input_size=151,seq_len=par.N,output_size=4080,hidden_dim=4080,n_layers=par.NUM_LAYER,drop_prob=0.5)
last_train=par.MODEL_EVAL
path_pretrained_model=os.path.join(par.LOG_DIR,"{last_train}/best_model.pkl".format(last_train=last_train))
#load_pretrained_model
model.load_state_dict(torch.load(path_pretrained_model))
model.cuda()
if (par.Eval_variable_legnth):
    predicted_tfeat=eval_variable_help(model,gallery_loader)
else:
    predicted_tfeat= eval_help(model,gallery_loader)

log_dir=os.path.join(par.LOG_DIR,"log_eval.txt")
#evaluate the top@k results
dim = 4080  # dimension
database = gallery_feat
queries = predicted_tfeat# Dipende dallo step di tempo


In [ ]:
num_query

In [ ]:
k = 10
print("N is :",par.N)
num_database = database.shape[0]
num_query = queries.shape[0]
print("Step:{s} ,Num of image in database is {n1}, Num of query img is {n2}".format(s=par.N,n1=num_database,n2=num_query))

index = faiss.IndexFlatL2(dim)
index.add(database)
_, knn = index.search(queries, k)
assert (query_labels.shape[0] == queries.shape[0])
num_foto=0
hits = 0


In [ ]:
import matplotlib.pyplot as plt
im1=os.path.join(par.ROOT_DIR,"dati/Images/Female/4_Shirts_blouses/0045884_16.jpg")
# read images from computer

import matplotlib.ticker as plticker
try:
    from PIL import Image,ImageOps
except ImportError:
    import Image,ImageOps


def plt_imges(imgs,inx,id):
    image = Image.open(imgs[0])
    my_dpi=300
    fig=plt.figure(figsize=(float(image.size[0])/my_dpi,float(image.size[1])/my_dpi),dpi=my_dpi)
    f, axarr = plt.subplots(1,11)
   
    for i in range(len(imgs)):
        image = Image.open(imgs[i])
        if(i==inx):
            image = ImageOps.expand(image,border=30,fill='black')
        axarr[i].axis('off')
        axarr[i].imshow(image)
    path="/andromeda/shared/reco-pomigliano/fashion/disentagledFeaturesExtractor/log/Img_results/{id}.png".format(id=id)
    plt.savefig(path)
        
plt_imges([im1,im1,im1,im1,im1,im1],3,5)

  

In [ ]:
hits=0
tq=range(num_query)
for q in tq: # itera i dati predicted_tfeat
    neighbours_idxs = knn[q]# gli indici dei k-feat più simili alla predicted_tfeat[q]
    imgs=[os.path.join(par.ROOT_DIR,"dati/Images/",img_data[q])]
    count=0
    found=False
    index_im=0
    for n_idx in neighbours_idxs:
        imgs.append(os.path.join(par.ROOT_DIR,"dati/Images/",img_data[n_idx]))
        if (test_labels[n_idx] == query_labels[q]).all():
            hits += 1
            found=True
            index_im=count+1
            print(count)
            
        count=count+1
    if(found):
        plt_imges(imgs,index_im,q)
    
    #tq.set_description("Num of hit {h}".format(h=hits))